## Searching the Catalog
<hr>

#### Import the `Instance` class.

In [ ]:
from opal.query import Instance
import os

 - `Instance` takes two arguments: `api_url` and `token`
  - `api_url` is populated by the `BE_URL` environment varaible if left blank.
  - `token` is populated by the `JUPYTERHUB_API_URL` environment variable if left blank.

In [ ]:
# be_url = "http://opalcatalog-be:9001/services/opal-catalog"
instance = Instance()

<br>  
  
### Search types:
<hr>

<strong>Full search with no filters:</strong>

In [ ]:
result = instance.search()

assert len(result.all()) > 0
len(result.all())

#### Search filtering

 - Fuzzy match using `._with()`

In [ ]:
result = instance._with("attr", "attr_3").search()

attr_only = result.reduce(lambda x: x['kind_metadata']['attr'])
assert all([ x in attr_only for x in ["attr_3", "attr_30"]])
attr_only

 - Now apply `._or()` to fuzzy match on "attr_6" additionally

In [ ]:
result = instance._with("attr", "attr_3")._or("attr", "attr_6").search()

attr_only = result.reduce(lambda x: x['kind_metadata']['attr'])
assert all([ x in attr_only for x in ["attr_3", "attr_30", "attr_6"]])
attr_only

 - Then apply `._not()` to exclude "attr_30" from the search

In [ ]:
result = instance._with("attr", "attr_3")._or("attr", "attr_6")._not("attr", "attr_30").search()

attr_only = result.reduce(lambda x: x['kind_metadata']['attr'])
assert all([ x in attr_only for x in ["attr_3", "attr_6"]])
attr_only

### Chaining filters: `._and(key, val)._or(key, val)._not(key, val)`
<hr>  

The order of the chain matters:
 - `a._and(b)._or(c)` would evaluate to `(a && b) || c`
 - `a._or(c)._and(b)` would evaluate to `(a || c) && b`

### Searching by kind_type
<hr>

<strong>Apply the type filter: `instance._type(kind_type)`</strong>

You can include this filter at any point in the chain.

 - _Before type filter_

In [ ]:
result = instance\
    ._with("attr", "attr_3")\
    ._or("attr", "attr_6")\
    ._not("attr", "attr_30")\
    .search()
type_only = result.reduce(lambda x: x.get("kind_type"))
assert all( x in type_only for x in ["type_2", "type_4"])
type_only

 - _After applying type filter_

In [ ]:
result = instance\
    ._with("attr", "attr_3")\
    ._or("attr", "attr_6")\
    ._not("attr", "attr_30")\
    ._type("type_2")\
    .search()
type_only = result.reduce(lambda x: x.get("kind_type"))
assert len(result.all()) == 1
assert all( x in type_only for x in ["type_2"])
type_only

<strong>Use `instance.one(id)` to get a singular instance by kind_id:</strong>

In [ ]:
result = instance.one("test_1")
assert len(result.all()) == 1
result.all()

### The `Result` class:

Whenever `instance.search()` is called, all records are returned as a `<Result>` object. This class is just a wrapper around the returned JSON data with some extra methods.
<hr>

<strong>Get all the records in List[dict] format:</strong>

In [ ]:
result = instance._with("attr", "attr").search()
result.all()

<strong>Quickly get all ids from a result set with `Result.ids()`</strong>

In [ ]:
ids_only = result.ids()
assert all([ "test" in x for x in ids_only ])
ids_only

<strong>Reduce a value from a result set with `Result.reduce(lambda x)`:</strong>

In [ ]:
attr_only = result.reduce(lambda x: x['kind_metadata']['attr'])
assert all([ "attr" in x for x in attr_only ])
attr_only

<strong>Further filter a result list with `Result.filter(lambda x)`:</strong>

In [ ]:
r_all = result.all()
r_filtered = result.filter(lambda x: x['kind_type'] == "type_2")

print("Before: ", len(r_all))
print("After: ", len(r_filtered))

assert len(r_filtered) == 1
assert isinstance(r_filtered[0], dict)
assert r_filtered[0]['kind_type'] == "type_2"

r_filtered[0]

In [ ]:
import opal.publish

res = instance._with("attr", "attr_").search()
for r in res.all():
    opal.publish.delete(r["kind_id"])